# NFL API and Prediction Charts for Quiniela

## Config

In [1]:
%load_ext autotime

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import nflapi
from datetime import datetime

## Read in Data

In [3]:
nfl = nflapi.NFL(ua="nflapi_quiniela")

In [4]:
preds = pd.read_csv('../data/quiniela_res.csv').sort_values(['season', 'week'], ascending=True)
preds['season'] = preds['season'].astype(int)
preds['week'] = preds['week'].astype(int)
preds = preds.rename(columns={'Home Team': 'home_team', 'Away Team':'away_team'})

teamsL = np.unique(preds['away_team'].unique().tolist() + preds['home_team'].unique().tolist())
print(np.sort(teamsL))
print(len(teamsL))
preds.head()

['ARI' 'ATL' 'BAL' 'BUF' 'CAR' 'CHI' 'CIN' 'CLE' 'DAL' 'DEN' 'DET' 'GB'
 'HOU' 'IND' 'JAX' 'KC' 'LA' 'LAC' 'LV' 'MIA' 'MIN' 'NE' 'NO' 'NYG' 'NYJ'
 'PHI' 'PIT' 'SEA' 'SF' 'TB' 'TEN' 'WAS']
32


,season,week,away_team,home_team,Gel,Hector,Emilio,Sonny
0,2003,1,ATL,DAL,DAL,ATL,DAL,DAL
1,2003,1,NE,BUF,BUF,BUF,NE,BUF
2,2003,1,JAX,CAR,JAX,JAX,JAX,CAR
3,2003,1,DEN,CIN,DEN,DEN,DEN,DEN
4,2003,1,ARI,DET,DET,ARI,DET,ARI


In [5]:
teams = pd.read_csv('../data/nfl_team_info_all.csv', index_col=0)
teams['season'] = teams['season'].astype(int)
team_dec = teams.groupby(['full_name', 'abbreviation']).size().reset_index(drop=False).set_index(['full_name'])['abbreviation'].to_dict()
teams.head()

,abbreviation,full_name,nick_name,conference,division,city_state_region,season,venue
0,WAS,Washington Redskins,Redskins,NFC,NFC_EASTERN,Washington,2003,NaN
1,TB,Tampa Bay Buccaneers,Buccaneers,NFC,NFC_SOUTH,Tampa Bay,2003,NaN
2,SEA,Seattle Seahawks,Seahawks,NFC,NFC_WESTERN,Seattle,2003,NaN
3,SF,San Francisco 49ers,49ers,NFC,NFC_WESTERN,San Francisco,2003,NaN
4,LAC,San Diego Chargers,Chargers,AFC,AFC_WESTERN,San Diego,2003,NaN


In [6]:
scores = pd.read_csv('../data/nfl_scores_all.csv', index_col=0)
scores['season'] = scores['season'].astype(int)
scores['week'] = scores['week'].astype(int)
scores['home_team'] = scores['home_team'].apply(lambda x: team_dec[x])
scores['away_team'] = scores['away_team'].apply(lambda x: team_dec[x])
scores['winner'] = [home_team if home_score + home_ot_score > away_score + away_ot_score else \
                    away_team if away_score + away_ot_score > home_score + home_ot_score else \
                    'TIE' for home_ot_score, away_ot_score, home_score, away_score, home_team, away_team in \
                    scores[['home_team_score', 'away_team_score', 
                            'home_points_overtime_total', 'visitor_points_overtime_total', 
                            'home_team', 'away_team']].values]
scores.head()

,season,week,week_type,game_id,game_details_id,game_time,game_date,home_team,away_team,home_team_score,...,visitor_points_overtime_total,home_points_q1,home_points_q2,home_points_q3,home_points_q4,visitor_points_q1,visitor_points_q2,visitor_points_q3,visitor_points_q4,winner
0,2003,1,REG,10012003-0904-00a6-ff62-12e477623f1a,10160000-0264-94ab-68f5-9ff371a986a3,2003-09-04 09:00:00+00:00,2003-09-04,WAS,NYJ,16.0,...,0.0,3.0,10.0,0.0,3.0,7.0,0.0,3.0,3.0,WAS
1,2003,1,REG,10012003-0907-0075-30bd-8e1167d1c6ac,10160000-0264-997c-6c87-72c644130bbf,2003-09-07 09:00:00+00:00,2003-09-07,DET,ARI,42.0,...,0.0,7.0,7.0,14.0,14.0,7.0,7.0,10.0,0.0,DET
2,2003,1,REG,10012003-0907-0126-f566-a59e19a70a3a,10160000-0264-97a8-19af-9168b28ada0a,2003-09-07 09:00:00+00:00,2003-09-07,CIN,DEN,10.0,...,0.0,0.0,3.0,0.0,7.0,3.0,17.0,7.0,3.0,DEN
3,2003,1,REG,10012003-0907-0277-e7c6-41b36cc4a1ce,10160000-0264-9839-6579-47a20841e644,2003-09-07 09:00:00+00:00,2003-09-07,CLE,IND,6.0,...,0.0,3.0,0.0,0.0,3.0,0.0,3.0,3.0,3.0,IND
4,2003,1,REG,10012003-0907-0388-64cf-dd9886d18dde,10160000-0265-04d2-fd24-f2f5b0964d49,2003-09-07 09:00:00+00:00,2003-09-07,PIT,BAL,34.0,...,0.0,6.0,7.0,14.0,7.0,0.0,0.0,7.0,8.0,PIT


In [7]:
## Merge Home Team Info
df = scores.merge(teams, 
                    left_on=['home_team', 'season'], 
                    right_on=['abbreviation', 'season'], 
                    suffixes=['', '_home'],
                    how='left')

## Merge Away Team Info
df = df.merge(teams.drop(['city_state_region', 'venue'], axis=1), 
                    left_on=['away_team', 'season'], 
                    right_on=['abbreviation', 'season'], 
                    suffixes=['', '_away'],
                    how='left')

## Create New Week Num Column
week_type_order = {'REG':1, 'WC':2, 'DIV':3, 'CONF':4, 'SB':5}
df['week_type_ord'] = df['week_type'].apply(lambda x: week_type_order[x])
week_dec = df.sort_values(['game_date', 'week_type_ord', 'season'], ascending=True).reset_index(drop=True).groupby(['season', 'week_type_ord', 'week']).size().reset_index(drop=False)
week_dec['week_num'] = week_dec.groupby(['season'])['season'].cumcount()+1
df = df.merge(week_dec.drop(0, axis=1), 
                    left_on=['season', 'week', 'week_type_ord'],
                    right_on=['season', 'week', 'week_type_ord'],
                    how='left')


## Merge Users Pred Info
# df = df.merge(preds.drop(['week'], axis=1), 
df = df.merge(preds, 
                    left_on=['home_team', 'away_team', 'season'], 
                    right_on=['home_team', 'away_team', 'season'],
                    how='left',
                    suffixes=['', '_quin'])

## Remove Duplicate Entries
dup_games = scores.groupby(['home_team', 'away_team', 'season']).size()
dup_games = dup_games[dup_games>1].reset_index(drop=False)
drop_idxL = []
for idx, dup_game in dup_games.iterrows():
    temp_dup = df.loc[((df['home_team'] == dup_game['home_team']) & (df['away_team'] == dup_game['away_team']) &  (df['season'] == dup_game['season']))].reset_index(drop=False)
    temp_dup['week_num_diff'] = abs(temp_dup['week_quin'] - temp_dup['week_num'])
    drop_idxL = drop_idxL + temp_dup.sort_values(['week_num_diff'], ascending=False).drop_duplicates(['game_id'], keep='first')['index'].values.tolist()
df = df.loc[~df.index.isin(drop_idxL)].reset_index(drop=True)

## Preview Data
print(df.columns.tolist())
df.head()

['season', 'week', 'week_type', 'game_id', 'game_details_id', 'game_time', 'game_date', 'home_team', 'away_team', 'home_team_score', 'away_team_score', 'home_points_overtime_total', 'visitor_points_overtime_total', 'home_points_q1', 'home_points_q2', 'home_points_q3', 'home_points_q4', 'visitor_points_q1', 'visitor_points_q2', 'visitor_points_q3', 'visitor_points_q4', 'winner', 'abbreviation', 'full_name', 'nick_name', 'conference', 'division', 'city_state_region', 'venue', 'abbreviation_away', 'full_name_away', 'nick_name_away', 'conference_away', 'division_away', 'week_type_ord', 'week_num', 'week_quin', 'Gel', 'Hector', 'Emilio', 'Sonny']


,season,week,week_type,game_id,game_details_id,game_time,game_date,home_team,away_team,home_team_score,...,nick_name_away,conference_away,division_away,week_type_ord,week_num,week_quin,Gel,Hector,Emilio,Sonny
0,2003,1,REG,10012003-0904-00a6-ff62-12e477623f1a,10160000-0264-94ab-68f5-9ff371a986a3,2003-09-04 09:00:00+00:00,2003-09-04,WAS,NYJ,16.0,...,Jets,AFC,AFC_EASTERN,1,1,1.0,WAS,WAS,NYJ,WAS
1,2003,1,REG,10012003-0907-0075-30bd-8e1167d1c6ac,10160000-0264-997c-6c87-72c644130bbf,2003-09-07 09:00:00+00:00,2003-09-07,DET,ARI,42.0,...,Cardinals,NFC,NFC_WESTERN,1,1,1.0,DET,ARI,DET,ARI
2,2003,1,REG,10012003-0907-0126-f566-a59e19a70a3a,10160000-0264-97a8-19af-9168b28ada0a,2003-09-07 09:00:00+00:00,2003-09-07,CIN,DEN,10.0,...,Broncos,AFC,AFC_WESTERN,1,1,1.0,DEN,DEN,DEN,DEN
3,2003,1,REG,10012003-0907-0277-e7c6-41b36cc4a1ce,10160000-0264-9839-6579-47a20841e644,2003-09-07 09:00:00+00:00,2003-09-07,CLE,IND,6.0,...,Colts,AFC,AFC_SOUTH,1,1,1.0,IND,CLE,IND,CLE
4,2003,1,REG,10012003-0907-0388-64cf-dd9886d18dde,10160000-0265-04d2-fd24-f2f5b0964d49,2003-09-07 09:00:00+00:00,2003-09-07,PIT,BAL,34.0,...,Ravens,AFC,AFC_NORTH,1,1,1.0,PIT,PIT,PIT,PIT


In [8]:
df.loc[((df['Gel'].isnull()) & (df['game_details_id'].notnull()))]

,season,week,week_type,game_id,game_details_id,game_time,game_date,home_team,away_team,home_team_score,...,nick_name_away,conference_away,division_away,week_type_ord,week_num,week_quin,Gel,Hector,Emilio,Sonny


In [9]:
scores.loc[scores['winner'].isnull()]

,season,week,week_type,game_id,game_details_id,game_time,game_date,home_team,away_team,home_team_score,...,visitor_points_overtime_total,home_points_q1,home_points_q2,home_points_q3,home_points_q4,visitor_points_q1,visitor_points_q2,visitor_points_q3,visitor_points_q4,winner
